In [2]:
# coding: utf-8
import numpy as np
import math
import numpy as np

def Setting(num):
    global HDNnum
    HDNnum = num
    global HDN
    HDN = [[0 for i in range(HDNnum)] for j in range(HDNnum)]
    HDN[0][0] = np.matrix([[0, 1], [0, 0]])
    for k in range(HDNnum - 1):
        for l in range(k + 1):
            HDN[k + 1][l] = np.matrix([[0 for i in range(pow(2, k + 2))] for j in range(pow(2, k + 2))])
            for i in range(pow(2, k + 1)):
                for j in range(pow(2, k + 1)):
                    HDN[k + 1][l][i, j] = HDN[k][l][i, j]
                    HDN[k + 1][l][i + pow(2, k + 1), j] = 0.
                    HDN[k + 1][l][i, j + pow(2, k + 1)] = 0.
                    HDN[k + 1][l][i + pow(2, k + 1), j + pow(2, k + 1)] = HDN[k][l][i, j]
        HDN[k + 1][k + 1] = np.matrix([[0 for i in range(pow(2, k + 2))] for j in range(pow(2, k + 2))])
        for i in range(pow(2, k + 1)):
            for j in range(pow(2, k + 1)):
                if i == j:
                    HDN[k + 1][k + 1][i, j + pow(2, k + 1)] = 1.
    global ID
    ID = np.matrix(np.eye(2**HDNnum))

def Deriv(Mat, k):
    if 0 <= k <= HDNnum:
        return Mat[0,2**k-1]
    else:
        return "error"

def prod(Mat1,Mat2):
    return np.dot(Mat1,Mat2)

def const():
    return ID


def Hini(x):
    val = x * ID
    for i in range(HDNnum):
        val += HDN[HDNnum-1][i]
    return  val


def Hsin(Mat):
    a = Mat[0,0]
    Q = Mat-a*ID
    val = math.sin(a)*ID
    for i in range(HDNnum):
        val += (Q**(i+1))/math.factorial(i+1)*(
            (-1)**(((i+1)//2)%4)*((i+2)%2)*math.sin(a)
            +(-1)**((i//2)%4)*(i+1)%2)*math.cos(a)
    return  val

def Hcos(Mat):
    a = Mat[0,0]
    Q = Mat-a*ID
    val = math.cos(a)*ID
    for i in range(HDNnum):
        val += (Q**(i + 1)) * (
                    pow(-1, ((i + 1) // 2) % 4) * ((i + 2) % 2) * math.cos(Mat[0, 0]) + pow(-1, ((i + 2) // 2) % 4) * (
                        (i + 1) % 2) * math.sin(Mat[0, 0])) / math.factorial(i + 1)
    return  val


def Hexp(Mat):
    val = math.exp(Mat[0, 0]) * ID
    for i in range(HDNnum):
        val += ((Mat - Mat[0, 0] * ID) ** (i + 1)) * math.exp(Mat[0, 0]) / math.factorial(i + 1)
    return  val


def Hlog(Mat):
    val = math.log(Mat[0, 0]) * ID
    for i in range(HDNnum):
        val += ((Mat - Mat[0, 0] * ID) ** (i + 1)) * (pow(-1, i)) * (Mat[0, 0] ** (-i - 1)) / (i + 1)
    return  val


def Hpow(Mat, m):
    coef = 1
    val = math.pow(Mat[0, 0], m) * ID
    for i in range(HDNnum):
        coef *= m-i
        val += coef*((Mat - Mat[0, 0] * ID) ** (i + 1)) * math.pow(Mat[0, 0], m - i - 1) / math.factorial(i + 1)
    return  val


def HiniMult(x,k,alpha):
    val = x * ID
    for i in range(HDNnum):
        if alpha[i] == k:
            val += HDN[HDNnum - 1][i]
    return  val

In [3]:
# Maximum numbers of derivatives
num_deriv=10

# Initial settings (construction of basis)
Setting(num_deriv)

## One-variable function ##
# f(x) = exp(sin(sqrt(x)+3*x))*log(cos^2(x)+0.5)
def Func(x):
    #return prod(Hexp(Hsin(Hpow(Hini(x),1/2)+3.*Hini(x))),Hlog(Hpow(Hcos(Hini(x)),2)+0.5*const()))
    return Hsin(Hini(x))

# value
x = 0


# 4th derivatives of Func on x
for i in range(num_deriv+1):
    print(i,"-derivative(s)=",Deriv(Func(x),i))

0 -derivative(s)= 0.0
1 -derivative(s)= 1.0
2 -derivative(s)= 0.0
3 -derivative(s)= 1.0
4 -derivative(s)= 0.0
5 -derivative(s)= 1.0
6 -derivative(s)= 0.0
7 -derivative(s)= 1.0
8 -derivative(s)= 0.0
9 -derivative(s)= 1.0
10 -derivative(s)= 0.0


In [4]:
## Multi-variables function ##
# multi-index for partial derivative
alpha = [0,0,1,2,2,0,0,1,2,2] # ∂x0∂x0∂x1∂x2∂x2∂x0∂x0∂x1∂x2∂x2


# f(x0,x1,x2) = exp(sin(sqrt(x0)+3*x1))*log(cos^2(x2)+0.5)
def MultiFunc(x0,x1,x2):
    return prod(Hexp(Hsin(Hpow(HiniMult(x0,0,alpha),1/2)+3.*HiniMult(x1,1,alpha))),Hlog(Hpow(Hcos(HiniMult(x2,2,alpha)),2)+0.5*const()))


# values
x0 = 0.5*math.pi
x1 = 0.25*math.pi
x2 = 2.


# 4th partial derivatives of MultiFunc on (x0,x1,x2)
for i in range(num_deriv+1):
    print(i,"-partial derivative(s)=",Deriv(MultiFunc(x0,x1,x2),i))


0 -partial derivative(s)= -0.2520792083298674
1 -partial derivative(s)= 0.08975533445924486
2 -partial derivative(s)= -0.04437827677571985
3 -partial derivative(s)= 0.10110387071845282
4 -partial derivative(s)= -0.28721335391505015
5 -partial derivative(s)= -0.17323536901715456
6 -partial derivative(s)= 0.39460365873505027
7 -partial derivative(s)= -0.9005205372151645
8 -partial derivative(s)= 6.832227147259347
9 -partial derivative(s)= -82.66890813871979
10 -partial derivative(s)= 211.68218867767294
